In [1]:
# Author: Kenneth J Gollaher
# CS340: Project Two ipynb file
# Date: 17APR2022

from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

# Importing CRUD Python file animal_shelter alsong with my class AnimalShelter
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# Username and password and CRUD Python module name
username = "accuser"
password = "Goll788951"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Added in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Insert of HTML image tag in the line below
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard', style={'color':'blue'}))),
    # Added unique identifier
    html.Center(html.B(html.H1('Kenneth J Gollaher - Project Two', style={'color':'green'}))),
    html.Hr(),
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    html.Div(
        
# Added in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
# Added interactive data table to make it user-friendly for your client - copied from milestone six 
# Allows 10 rows per page, and starts on page 0. 
# Columns and Rows may be selected but not deleted/edited for aacuser access
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,

        style_header={
        'backgroundColor': 'rgb(230, 230, 230)',
        'fontWeight': 'bold',
        'text-transform': 'capitalize'
        },

    ),
    html.Br(),
    html.Hr(),
# Sets up the dashboard so that the chart and the geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
# Added code to filter interactive data table with MongoDB queries
        # Water Rescue filter
        if filter_type == 'water':
            df = pd.DataFrame.from_records(shelter.read_all({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
        # Mountain or Wilderness Rescue filter
        elif filter_type == 'mount':
            df = pd.DataFrame.from_records(shelter.read_all({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        # Disaster Rescue or Individual Tracking filter
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(shelter.read_all({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        # Reset to no filter
        else:
            df = pd.DataFrame.from_records(shelter.read_all({}))
        

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

# Background color change of selected columns by user
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Chart creation
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    # Added code for chart of choice (histogram was chosen)
    dff = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.histogram(dff, x='breed')
        )    
    ]

# Map creation
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):

    # Added code for your geolocation chart - copied from milestone six
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app